# 本次研讨课内容:
   问答摘要内容讲解:
    1. beam search 讲解.
    2. seq2seq baseline 编写\训练\测试.
    3. 提交你的第一版成绩.
    4. 预测结果分析,模型提升改进点讨论

In [149]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [151]:
def config_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)

In [152]:
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [ ]:
1. nvidia驱动   nvidia-smi
2. CUDA

# 休息10分钟! 9:10分见

In [ ]:
去除标点符号  200
不去除  500

In [ ]:
batch size 32
max_x_len 200

In [ ]:
rtx 2080

In [ ]:
10轮/分钟  5轮 25分

![](https://img-blog.csdn.net/20180414103300419)

解码是seq2seq模型的常见问题，常用方法有贪心搜索`（Greedy Search）`集束搜索`（Beam Search）`。

Decoder根据Encoder的中间语义编码向量c和`<s>`标签得到第一个输出的概率分布`[0.1,0.1,0.3,0.4,0.1]`，选择概率最大的`0.4`，即`moi`。

根据隐向量h1h1和moi得到第二个输出的概率分布`[0.1,0.1,0.1,0.1,0.6][0.1,0.1,0.1,0.1,0.6]`，选择概率最大的`0.6`，即`suis`。

以此类推，直到遇到`<\s>`标签，得到最终的序列`moi suis étudiant`.

# 集束搜索

上面的贪心搜索只选择了概率最大的一个，而集束搜索则选择了概率最大的前k个。这个k值也叫做集束宽度（Beam Width）。

还是以上面的例子作为说明，k值等于2，则集束搜索的过程如下图：

![](https://img-blog.csdn.net/20180414113522371?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L2d1b2xpbmRvbmdnbGQ=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

得到第一个输出的概率分布`[0.1,0.1,0.3,0.4,0.1] [0.1,0.1,0.3,0.4,0.1]`，选择概率最大的前两个，`0.3`和`0.4`，即`Je`和`moi`。

然后`Je`和`moi`分别作为`Decoder`的输入，得到两个概率分布，然后再选择概率和最大的前两个序列，`0.3+0.8`和`0.4+0.6`，即`Je suis`和`moi suis`。

以此类推，最终可以得到两个序列，即`Je suis étudiant`和`moi suis étudiant`，很明显前者的概率和最大，为`2.2`，所以这个序列是最终得到的结果。

集束搜索本质上也是贪心的思想，只不过它考虑了更多的候选搜索空间，因此可以得到更多的翻译结果。

![](https://img-blog.csdn.net/20181011144011354?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzE2MjM0NjEz/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
集束搜索可以认为是维特比算法的贪心形式，在维特比所有中由于利用动态规划导致当字典较大时效率低，而集束搜索使用beam size参数来限制在每一步保留下来的可能性词的数量。集束搜索是在测试阶段为了获得更好准确性而采取的一种策略，在训练阶段无需使用。

假设字典为[a,b,c]，beam size选择2，则如下图有：

1：在生成第1个词的时候，选择概率最大的2个词，那么当前序列就是a或b

2：生成第2个词的时候，我们将当前序列a或b，分别与字典中的所有词进行组合，得到新的6个序列aa ab ac ba bb bc,然后从其中选择2个概率最高的，作为当前序列，即ab或bb

3：不断重复这个过程，直到遇到结束符为止。最终输出2个概率最高的序列。

# 加载训练好的模型

In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/home/roger/kaikeba/03_lecture/code')
from utils.wv_loader import load_vocab
from utils.data_loader import load_dataset
from utils.config import *
import numpy as np
from utils.gpu_utils import config_gpu
config_gpu()
import tensorflow as tf
from seq2seq_tf2.seq2seq_model import Seq2Seq

1 Physical GPUs, 1 Logical GPUs


In [113]:
vocab, reverse_vocab = load_vocab(vocab_path)
batch_size=3
vocab_size=len(vocab)
params = {}
params["vocab_size"] = vocab_size
params["embed_size"] = 500
params["enc_units"] = 512
params["attn_units"] = 512
params["dec_units"] = 512
params["batch_size"] = batch_size
params["beam_size"]=3

params['min_dec_steps']=4
params['max_dec_steps']=50

In [91]:
train_X,train_Y,test_X = load_dataset()

# 载入模型

In [93]:
params

{'vocab_size': 31819,
 'embed_size': 500,
 'enc_units': 512,
 'attn_units': 512,
 'dec_units': 512,
 'batch_size': 32,
 'beam_size': 3,
 'min_dec_steps': 4,
 'max_dec_steps': 50}

In [108]:
# model = Seq2Seq(params)

In [95]:
model

# 2 读取训练好的模型

In [96]:
from utils.config import checkpoint_dir,checkpoint_prefix

In [97]:
ckpt = tf.train.Checkpoint(Seq2Seq=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)
ckpt.restore(ckpt_manager.latest_checkpoint)
print("Model restored")

Model restored


# code 

In [98]:
class Hypothesis:
    """ Class designed to hold hypothesises throughout the beamSearch decoding """
    def __init__(self, tokens, log_probs, hidden, attn_dists):
        self.tokens = tokens  # list of all the tokens from time 0 to the current time step t
        self.log_probs = log_probs  # list of the log probabilities of the tokens of the tokens
        self.hidden = hidden  # decoder hidden state after the last token decoding
        self.attn_dists = attn_dists  # attention dists of all the tokens
        self.abstract = ""

    def extend(self, token, log_prob, hidden, attn_dist):
        """Method to extend the current hypothesis by adding the next decoded token and all the informations associated with it"""
        return Hypothesis(tokens=self.tokens + [token],  # we add the decoded token
                          log_probs=self.log_probs + [log_prob],  # we add the log prob of the decoded token
                          hidden=hidden,  # we update the state
                          attn_dists=self.attn_dists + [attn_dist])
    @property
    def latest_token(self):
        return self.tokens[-1]

    @property
    def tot_log_prob(self):
        return sum(self.log_probs)

    @property
    def avg_log_prob(self):
        return self.tot_log_prob / len(self.tokens)

# 初始化一个对象列表

In [99]:
batch_size

32

In [105]:
enc_inp=test_X[:3]

In [106]:
enc_inp.shape

(3, 200)

In [107]:
enc_output, enc_hidden = model.call_encoder(enc_inp)

In [109]:
enc_output.sdhape

TensorShape([3, 200, 512])

In [110]:
enc_hidden.shape

TensorShape([3, 512])

In [114]:
hyps = [Hypothesis(tokens=[vocab['<START>']],
                   log_probs=[0.0],
                   hidden=enc_hidden[0],
                   attn_dists=[],
                   ) for _ in range(params['batch_size'])]

In [ ]:
vocab {a c f e p}

In [ ]:
token [c f p]

In [ ]:
<START> 

In [115]:
hyps

In [49]:
results = []  # list to hold the top beam_size hypothesises
steps = 0  # initial step

## 获取最新tokens 

In [116]:
latest_tokens = [h.latest_token for h in hyps]

In [117]:
latest_tokens

[31815, 31815, 31815]

## 隐藏层状态

In [118]:
hiddens = [h.hidden for h in hyps]

In [119]:
len(hiddens)

3

![](https://img-blog.csdn.net/20180414103300419)

## 单步运行decode

In [122]:
pad_index = vocab['<PAD>']
nuk_index = vocab['<UNK>']
start_index = vocab['<START>']
stop_index = vocab['<STOP>']

In [123]:
# 第一轮
steps=0

In [124]:
def decoder_onestep(enc_output,dec_input,dec_hidden):
    # 单个时间步 运行
    preds, dec_hidden, context_vector,attention_weights = model.call_decoder_onestep(dec_input,dec_hidden, enc_output)
    # 拿到top k个index 和 概率
    top_k_probs, top_k_ids = tf.nn.top_k(tf.squeeze(preds), k=params["beam_size"])
    # 计算log概率
    top_k_log_probs = tf.math.log(top_k_probs)
    # 返回需要保存的中间结果和概率
    return preds,dec_hidden,context_vector,attention_weights,top_k_log_probs,top_k_ids

In [ ]:
(1,vocab_size)

## 单次搜索

In [125]:
# 计算第encoder的输出
enc_output, enc_hidden = model.call_encoder(enc_inp)

In [127]:
# 第一个decoder输入 开始标签
dec_input = tf.expand_dims(latest_tokens, 1)

In [129]:
# 第一个隐藏层输入
dec_hidden = enc_hidden

In [130]:
# 单步运行
preds, dec_hidden, context_vector,attention_weights, top_k_log_probs, top_k_ids = decoder_onestep(enc_output,dec_input,dec_hidden)

In [137]:
# 现阶段全部可能情况
all_hyps = []
# 原有的可能情况数量
num_orig_hyps = 1 if steps == 0 else len(hyps)

# 遍历添加所有可能结果
for i in range(num_orig_hyps):
    h, new_hidden, attn_dist = hyps[i], dec_hidden[i], attention_weights[i]
    # 分裂 添加 beam size 种可能性
    for j in range(params['beam_size']):
        # 构造可能的情况
        new_hyp = h.extend(token = top_k_ids[i, j].numpy(),
                                       log_prob = top_k_log_probs[i, j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
        # 添加可能情况
        all_hyps.append(new_hyp)

In [ ]:
token c b f 

In [ ]:
1. <start> c  
2. <start> b
3. <start> f

In [139]:
all_hyps

## 排序

In [140]:
# 重置
hyps = []

In [142]:
# 按照概率来排序
sorted_hyps = sorted(all_hyps, key=lambda h: h.avg_log_prob, reverse=True)

In [66]:
# 筛选top前beam_size句话 top 3
for h in sorted_hyps:
    if h.latest_token == stop_index:
        # 长度符合预期,遇到句尾,添加到结果集
        if steps >= params['min_dec_steps']:
            results.append(h)
    else:
        # 未到结束 ,添加到假设集
        hyps.append(h)
    
    # 如果假设句子正好等于beam_size 或者结果集正好等于beam_size 就不在添加
    if len(hyps) == params['beam_size'] or len(results) == params['beam_size']:
        break
steps += 1

In [67]:
if len(results) == 0:
    results = hyps

In [ ]:
avg_log_prob

In [144]:
hyps_sorted = sorted(results, key=lambda h: h.avg_log_prob, reverse=True)

In [145]:
best_hyp = hyps_sorted[0]

In [147]:
best_hyp.abstract = " ".join([reverse_vocab[index] for index in best_hyp.tokens])

In [148]:
best_hyp.abstract

'<START> 捡'

References

[1] https://www.tensorflow.org/tutorials/seq2seq

[2] https://blog.csdn.net/guolindonggld/article/details/79938567

# beam search 方法整合

In [79]:
def beam_decode(model,batch,vocab, params):
    # 初始化mask
    start_index = vocab['<START>']
    stop_index = vocab['<STOP>']
    
    batch_size= params['batch_size']
    
    # 单步decoder
    def decoder_onestep(enc_output,dec_input,dec_hidden):
        # 单个时间步 运行
        preds, dec_hidden, context_vector,attention_weights = model.call_decoder_onestep(dec_input,dec_hidden, enc_output)
        # 拿到top k个index 和 概率
        top_k_probs, top_k_ids = tf.nn.top_k(tf.squeeze(preds), k=params["beam_size"])
        # 计算log概率
        top_k_log_probs = - tf.math.log(top_k_probs)
        # 返回需要保存的中间结果和概率
        return preds,dec_hidden,context_vector,attention_weights,top_k_log_probs,top_k_ids
    
    # 计算第encoder的输出
    enc_output, enc_hidden = model.call_encoder(batch)
    
    # 初始化batch size个 假设对象
    hyps = [Hypothesis(tokens=[start_index],
                   log_probs=[0.0],
                   hidden=enc_hidden[0],
                   attn_dists=[],
                   ) for _ in range(batch_size)]
    # 初始化结果集
    results = []  # list to hold the top beam_size hypothesises
    # 遍历步数
    steps = 0  # initial step
    
    # 第一个隐藏层输入
    dec_hidden = enc_hidden
    
    
    # 长度还不够 并且 结果还不够 继续搜索
    while steps < params['max_dec_steps'] and len(results) < params['beam_size']:
        # 获取最新待使用的token
        latest_tokens = [h.latest_token for h in hyps]
        # 获取所以隐藏层状态
        hiddens = [h.hidden for h in hyps]
        # 最新输入
        dec_input = tf.expand_dims(latest_tokens, 1)
        
        # 单步运行decoder 计算需要的值
        preds, dec_hidden, context_vector,attention_weights, top_k_log_probs, top_k_ids = decoder_onestep(enc_output,dec_input,dec_hidden)
        
        # 现阶段全部可能情况
        all_hyps = []
        # 原有的可能情况数量
        num_orig_hyps = 1 if steps == 0 else len(hyps)

        # 遍历添加所有可能结果
        for i in range(num_orig_hyps):
            h, new_hidden, attn_dist = hyps[i], dec_hidden[i], attention_weights[i]
            # 分裂 添加 beam size 种可能性
            for j in range(params['beam_size']):
                if params['batch_size']==1:
                    # 构造可能的情况
                    new_hyp = h.extend(token = top_k_ids[j].numpy(),
                                       log_prob = top_k_log_probs[j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
                else:
                    # 构造可能的情况
                    new_hyp = h.extend(token = top_k_ids[i, j].numpy(),
                                       log_prob = top_k_log_probs[i, j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
                # 添加可能情况
                all_hyps.append(new_hyp)
        
        # 重置
        hyps = []
        # 按照概率来排序
        sorted_hyps = sorted(all_hyps, key=lambda h: h.avg_log_prob, reverse=True)
        
        # 筛选top前beam_size句话
        for h in sorted_hyps:
            if h.latest_token == stop_index:
                # 长度符合预期,遇到句尾,添加到结果集
                if steps >= params['min_dec_steps']:
                    results.append(h)
            else:
                # 未到结束 ,添加到假设集
                hyps.append(h)

            # 如果假设句子正好等于beam_size 或者结果集正好等于beam_size 就不在添加
            if len(hyps) == params['beam_size'] or len(results) == params['beam_size']:
                break

        steps += 1
        
    if len(results) == 0:
        results = hyps
    
    hyps_sorted = sorted(results, key=lambda h: h.avg_log_prob, reverse=True)
    best_hyp = hyps_sorted[0]
    best_hyp.abstract = " ".join([reverse_vocab[index] for index in best_hyp.tokens])
    return best_hyp

In [81]:
# 读取模型
model = Seq2Seq(params)

In [82]:
# 构造数据
test_batch=test_X[:batch_size]

In [83]:
test_batch.shape

(3, 200)

In [84]:
# 获得最好的语句
best_hyp=beam_decode(model,test_batch,vocab, params)

In [85]:
best_hyp.abstract

'<START> 左高右 难关 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段 齿合器 某一 第一段'